In [2]:
initial_bays = [[(3, "heavy"), ], [(1, "light"), (6, "light"),], [], [(4, "light"),  (2, "heavy"), (5, "light")], [], []]
initial_crane_position = 5
initial_crane_container_held = 0
initial_cost = 0

initial_state = ([initial_bays, initial_crane_position, initial_crane_container_held], 0)

The environment sees a set of loading_bays, where a set of containers are positioned, and a crane.
The initial state will include: - The initial set up of the containers in the bays, 
                                - The location of the crane 
                                - What container it is hoding if any